In [1]:
from acquire import acquire_data
from wrangle import wrangle_data

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import minmax_scale

# Aqcuisition

In [2]:
df = acquire_data()

csv has been previously generated.
Data Acquired


In [3]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,fips,FIPS,Name,State
0,0,3.0,2.0,1458.0,136104.0,6037.0,6037,Los Angeles,CA
1,1,2.0,1.0,1421.0,35606.0,6037.0,6037,Los Angeles,CA
2,2,3.0,2.0,1650.0,614000.0,6037.0,6037,Los Angeles,CA
3,3,2.0,1.0,693.0,274237.0,6037.0,6037,Los Angeles,CA
4,4,0.0,0.0,1378.0,168828.0,6037.0,6037,Los Angeles,CA


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15036 entries, 0 to 15035
Data columns (total 9 columns):
Unnamed: 0                      15036 non-null int64
bedroomcnt                      15036 non-null float64
bathroomcnt                     15036 non-null float64
calculatedfinishedsquarefeet    15012 non-null float64
taxvaluedollarcnt               15036 non-null float64
fips                            15036 non-null float64
FIPS                            15036 non-null int64
Name                            15036 non-null object
State                           15036 non-null object
dtypes: float64(5), int64(2), object(2)
memory usage: 1.0+ MB


In [5]:
print(f'Total number of rows: {len(df)}')

Total number of rows: 15036


# Preparation

In [6]:
train, test = wrangle_data(df)

Data Prepared


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
print(f'''
Dropped all rows which contained a zero.
Result: {15036 - 14990} rows dropped. Total is now {len(df)} rows''')

# Exploration

# Now that a csv has been generated and the data has been pulled and cleaned, we can start exploring it.
Goal is to try and find insight into what's going on with the data through graphs.

# Scale the Data

Since there is a large disparity in terms of things like number of bathrooms/bedrooms compared to the price of the home, we should probably scale the data

In [ ]:
x = df[['total_sqft', 'bedroom_count', 'bathroom_count']]

In [ ]:
# Generate the minmax scaler
scale = minmax_scale(x)

In [ ]:
y = df.tax_amount

In [ ]:
plt.figure(figsize=(14,10))

# Left graph
plt.subplot(131)
sns.scatterplot(df.total_sqft, y)
plt.title('Total Square Feet')
plt.xlabel('Total Square Feet')
plt.ylabel('Tax Amount')

# Center graph
plt.subplot(132)
sns.scatterplot(df.bedroom_count, y)
plt.title('Number of Bedrooms')
plt.xlabel('# of Bedrooms')
plt.ylabel('Tax Amount')

# Right graph
plt.subplot(133)
sns.scatterplot(df.bathroom_count, y)
plt.title('Number of Bathrooms')
plt.xlabel('# of Bathrooms')
plt.ylabel('Tax Amount')

plt.suptitle('What Affects Taxes Most')